In [ ]:
import subprocess
import os
import sys

def check_requirements():
    """Check if required packages are installed"""
    try:
        # Check if jupyter is available
        result = subprocess.run(['jupyter', '--version'], capture_output=True, text=True)
        print(f"✅ Jupyter version: {result.stdout.strip()}")
        
        # Check if nbconvert is available
        result = subprocess.run(['jupyter', 'nbconvert', '--version'], capture_output=True, text=True)
        print(f"✅ nbconvert version: {result.stdout.strip()}")
        
        return True
    except FileNotFoundError:
        print("❌ Jupyter not found. Install with: pip install jupyter")
        return False
    except Exception as e:
        print(f"❌ Error checking requirements: {e}")
        return False

def notebook_to_word(notebook_path, output_path=None):
    """Convert notebook to Word document with better error handling"""
    
    # Check if notebook file exists
    if not os.path.exists(notebook_path):
        print(f"❌ Notebook file not found: {notebook_path}")
        return False
    
    if output_path is None:
        output_path = notebook_path.replace('.ipynb', '.docx')
    
    try:
        # First try with detailed error output
        result = subprocess.run([
            'jupyter', 'nbconvert', 
            '--to', 'docx', 
            notebook_path,
            '--output', output_path
        ], capture_output=True, text=True, check=True)
        
        print(f"✅ Notebook converted to: {output_path}")
        if result.stdout:
            print(f"Output: {result.stdout}")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Error converting notebook:")
        print(f"Return code: {e.returncode}")
        print(f"Error output: {e.stderr}")
        if e.stdout:
            print(f"Standard output: {e.stdout}")
        return False
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return False

def try_alternative_formats(notebook_path):
    """Try converting to other formats as alternatives"""
    formats = ['html', 'pdf', 'markdown']
    
    for fmt in formats:
        try:
            output_path = notebook_path.replace('.ipynb', f'.{fmt}')
            result = subprocess.run([
                'jupyter', 'nbconvert', 
                '--to', fmt, 
                notebook_path,
                '--output', output_path
            ], capture_output=True, text=True, check=True)
            
            print(f"✅ Successfully converted to {fmt}: {output_path}")
            return True
            
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to convert to {fmt}: {e.stderr}")
            continue
    
    return False



# Generate the HTML for the full report
print("🔍 Checking requirements...")
if check_requirements():
    print("\n📄 Converting notebook...")
    success = notebook_to_word('001_v7_end_to_end_churn_modeling_workflow.ipynb', 
                              'REPORT_v7_churn_models_and_analyis.docx')
    
    if not success:
        print("\n🔄 Trying alternative formats...")
        try_alternative_formats('001_v7_end_to_end_churn_modeling_workflow.ipynb')
else:
    print("❌ Requirements check failed. Please install missing packages.")